**AUGMENTED REALITY**

**Functions**

In [ ]:
# Calculando FPS
# Use -> start_time = time.time() # start time of the loop
def CalcFPS(start_time):
    fps = 'FPS: ' + str(int(1 / ((time.time() - start_time)/20)))
    
    return fps

# Load previously saved data of calibration
def LoadCalib():
    with np.load('data_calib_left.npz') as X:
        mtxL, distL, rvecsL, tvecsL, objpointsL, imgpointsL = [X[i] for i in ('mtxL', 'distL', 'rvecsL', 'tvecsL', 'objpointsL', 'imgpointsL')]
    with np.load('data_calib_right.npz') as X:
        mtxR, distR, rvecsR, tvecsR, objpointsR, imgpointsR = [X[i] for i in ('mtxR', 'distR', 'rvecsR', 'tvecsR', 'objpointsR', 'imgpointsR')]

    return mtxL, distL, rvecsL, tvecsL, objpointsL, imgpointsL, mtxR, distR, rvecsR, tvecsR, objpointsR, imgpointsR

# Apply Undistortion -- optional use
def ImageUndistort(img, mtx, dist):
    h,  w = img.shape[:2]
    newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

    # Using undistort
    dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

    # crop the image
    x,y,w,h = roi
    dst = dst[y:y+h, x:x+w]
    
    return dst

# Re-projection Errors -- optional use
def ProjError(imgpoints, objpoints, rvecs, tvecs, mtx, dist):
    tot_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
        tot_error += error
    
    errors = tot_error/len(objpoints)
    return errors

# Draw xyz axes -- other option of drawing
def DrawAxes(img, corners, rvecs, tvecs, mtx, dist):
    axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
    
    # project 3D points to image plane
    imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)
    
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

# Draw Cube at axes
def DrawCube(img, corners, rvecs, tvecs, mtx, dist):
    axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])
    
    # project 3D points to image plane
    imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)  
    imgpts = np.int32(imgpts).reshape(-1,2)

    # draw ground floor in green
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),-3)

    # draw pillars in blue color
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)

    # draw top layer in red color
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)

    return img

print('Functions OK')

**Main program**

In [ ]:
# POSE ESTIMATION
import numpy as np
import cv2
import time

pt_col = 6
pt_lin = 5
count_time = 0
font = cv2.FONT_HERSHEY_SIMPLEX
fps = 'FPS: 30'

# Open cameras
CamR= cv2.VideoCapture(1)
CamL= cv2.VideoCapture(0)

# Load all calibration data
mtxL, distL, rvecsL, tvecsL, objpointsL, imgpointsL, mtxR, distR, rvecsR, tvecsR, objpointsR, imgpointsR = LoadCalib()
print('Data of calibration loaded with sucess!')

if CamR.isOpened() and CamL.isOpened():
    print ('CAMERA ON')
else:
    print('CAMERA OFF')

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((pt_lin*pt_col,3), np.float32)
objp[:,:2] = np.mgrid[0:pt_lin,0:pt_col].T.reshape(-1,2)
    
while(True):
    if count_time == 0:
        start_time = time.time()
    
    retL, frameL= CamL.read()

    grayL = cv2.cvtColor(frameL, cv2.COLOR_BGR2GRAY)
        
    retL, cornersL = cv2.findChessboardCorners(grayL, (pt_lin,pt_col),None)

    if retL == True:
        cornersL2 = cv2.cornerSubPix(grayL,cornersL,(11,11),(-1,-1),criteria)

        # Find the rotation and translation vectors.
        retvals, rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, cornersL2, mtxL, distL)

        frameL = DrawCube(frameL, cornersL2, rvecs, tvecs, mtxL, distL)
    
    # Calcule FPS rate and draw
    count_time += 1
    if count_time == 20:
        fps = CalcFPS(start_time)
        count_time = 0
    cv2.putText(frameL, fps, (10,20), font, 0.5, (255,255,255), 1, cv2.LINE_AA)

    cv2.imshow('frameL',frameL)
    
    # End the Programme
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# When everything done, release the capture
CamR.release()
CamL.release()
cv2.destroyAllWindows()